# Deep Learning: CNN with ASL dataset
> AHSNCCU/NTNU CSIE 王修佑

Convolution Neural Network 卷積神經網路

## 載入和準備資料

In [2]:
import tensorflow.keras as keras
import pandas as pd

# Load in our data from CSV files
train_df = pd.read_csv("asl/sign_mnist_train.csv")
valid_df = pd.read_csv("asl/sign_mnist_valid.csv")

# Separate out our target values
y_train = train_df['label']
y_valid = valid_df['label']
del train_df['label']
del valid_df['label']

# Separate out our image vectors
x_train = train_df.values
x_valid = valid_df.values

# Turn our scalar targets into binary categories
num_classes = 24
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

# Normalize our image data
x_train = x_train / 255
x_valid = x_valid / 255

## 為卷積神經網路重新調整影像

+ 將目前的形狀(784像素) `(27455, 784)`轉換成(28x28像素) `(27455, 28, 28, 1)`
+ reshape(`-1`不改變維度, `28`像素, `28`像素, `1`色頻數量)

In [3]:
x_train.shape, x_valid.shape

((27455, 784), (7172, 784))

In [ ]:
x_train = x_train.reshape(-1,28,28,1)
x_valid = x_valid.reshape(-1,28,28,1)

In [ ]:
x_train.shape

In [ ]:
x_valid.shape

In [ ]:
x_train.shape, x_valid.shape

## 建立卷積模型

<img src="images/cnn.png" width=180 />

解決類似的問題，可使用開源模型修改

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)

model = Sequential()
model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu", 
                 input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(25, (3, 3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Flatten())
model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(units=num_classes, activation="softmax"))

### Conv2D

<img src="images/conv2d.png" width=300 />

2D 卷積層
```Python
model.add(Conv2D(75 , (3,3) , strides = 1 , padding = 'same'...)
```
75 指的是將要學習的核心(過濾器filter)數量。(3,3) 指的是這些核心的大小。strides指的是濾鏡經過影像時採用的步長。padding指的是從濾鏡建立的輸出影像是否與輸入影像的大小相符(convolution 會縮小)。

<img src="images/filter.gif" width=600 />

### BatchNormalization

批次正規化
> 調整隱藏層中的值，以提升訓練品質<br>
+ 有效減緩Activation Function(Sigmoid) 梯度(Gradient)消失的問題
+ 解決 Internal Covariate Shift
+ 加速模型收斂
+ 具有正則化效果

> Reference: https://medium.com/ching-i/batch-normalization-%E4%BB%8B%E7%B4%B9-135a24928f12